In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns  #for EDA

import re #  for text cleaning


from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from wordcloud import WordCloud
from collections import Counter

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Carregando Bancos:


In [28]:
fake = pd.read_csv("C:/Users/arthur.g.souza/Downloads/real.csv/fake.csv")
real = pd.read_csv("C:/Users/arthur.g.souza/Downloads/real.csv/real.csv")


Tirando duplicatas:

In [29]:
real=real.drop_duplicates()
fake=fake.drop_duplicates()

Criando etiquetas

In [30]:
real["etiqueta"] = 1
fake["etiqueta"] = 0

Juntando os bancos:

In [31]:
banco = pd.concat([real, fake], axis = 0)

Tornando tweet como string

In [32]:
banco["tweet"] = banco["tweet"].astype(str)

Tratando as palavras:


In [33]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\arthur.g.souza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\arthur.g.souza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [34]:
#converting into the lowercase
def lowercase(text):
    text=text.lower()
    return text

banco["tweet"] = banco["tweet"].apply(lowercase)

In [35]:
def text_clean(text):
    
    text=re.sub("\[.*#&]"," ",text)
    
    text=re.sub("https?://\S+|www\.\S+","",text)
    
    text=re.sub("[^a-zA-Z0-9\s]+","",text)
    
    text = re.sub('\w*\d\w*', '', text)
    
    stop_words = set(stopwords.words('english'))
    words = text.split()
    
    
    filtered_words = [word for word in words if word not in stop_words]
    text = ' '.join(filtered_words)

    # Remove extra whitespace
    text = re.sub('\s+', ' ', text).strip()

   
    return text
    
banco["tweet"] = banco["tweet"].apply(text_clean)    

Gerando Tokens para as palavras:

In [36]:
from nltk.stem import SnowballStemmer

# initialize SnowballStemmer 
stemmer = SnowballStemmer('english')

def stem_text(text):
    # Tokenize the input text into individual words
    tokens = nltk.word_tokenize(text)
    
    # Stem each token using the SnowballStemmer
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    
    # Join the stemmed tokens back into a single string
    return ' '.join(stemmed_tokens)

In [37]:
# apply stemming to the 'text' column in your DataFrame
banco["tweet"] = banco["tweet"].apply(stem_text)

Criando o modelo:

In [42]:
# Define TF-IDF vectorizer
vectorizer = TfidfVectorizer()

# Vectorize the text data
x = vectorizer.fit_transform(banco["tweet"])
y = banco["etiqueta"]

x_treino, x_teste, y_treino, y_teste = train_test_split(x, y, test_size = 0.3)


Usando Naive Byes/Multinomial

In [44]:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB().fit(x_treino, y_treino)

nb_model.score(x_teste,y_teste)

0.9420313520574788

In [48]:
y_pred = nb_model.predict(x_teste)

In [49]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_teste, y_pred))

[[5546  322]
 [ 388 5992]]


Usando RandomForest

In [47]:
from sklearn.ensemble import RandomForestClassifier

rfc=RandomForestClassifier()

rfc.fit(x_treino, y_treino)

rfc.score(x_teste,y_teste)

0.9451338994121489

In [50]:
ypred = rfc.predict(x_teste)
print(confusion_matrix(y_teste, ypred))

[[5542  326]
 [ 346 6034]]
